<a href="https://colab.research.google.com/github/likhon15-4915/Fraud-Detection-DMML/blob/main/fraud_detection_isolation_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


!pip install pandas numpy scikit-learn tensorflow



In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import classification_report, roc_auc_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [ ]:
# Load the dataset (ensure to upload the file or provide the correct path)
data = pd.read_csv('/content/drive/MyDrive/data mining lab/Synthetic_Financial_datasets_log.csv')

# Display the first few rows
data.head()



,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/data mining lab/Synthetic_Financial_datasets_log.csv")
# Explore the dataset
print("Dataset Info:")
print(df.info())
print("\nClass Distribution:")
print(df['isFraud'].value_counts())

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB
None

Class Distribution:
isFraud
0    6354407
1       8213
Name: count, dtype: int64


In [ ]:
# Convert categorical 'type' column to numeric using one-hot encoding
X = pd.get_dummies(df.drop(['isFraud', 'isFlaggedFraud', 'step', 'nameOrig', 'nameDest'], axis=1), columns=['type'])

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, df['isFraud'], test_size=0.2, random_state=42)


In [ ]:
# Isolation Forest for Anomaly Detection
print("\nRunning Isolation Forest...")
iso_forest = IsolationForest(contamination=0.01, random_state=42)
iso_forest.fit(X_train)
y_pred_iso = iso_forest.predict(X_test)
y_pred_iso = [1 if x == -1 else 0 for x in y_pred_iso]

# Evaluate Isolation Forest
print("\nIsolation Forest Results:")
print(classification_report(y_test, y_pred_iso))
print("AUC Score:", roc_auc_score(y_test, y_pred_iso))



Running Isolation Forest...

Isolation Forest Results:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99   1270904
           1       0.01      0.05      0.01      1620

    accuracy                           0.99   1272524
   macro avg       0.50      0.52      0.50   1272524
weighted avg       1.00      0.99      0.99   1272524

AUC Score: 0.5218609162658439


In [ ]:
# Define the Autoencoder model
autoencoder = Sequential()
autoencoder.add(Dense(14, activation='relu', input_shape=(X_train.shape[1],)))
autoencoder.add(Dense(7, activation='relu'))
autoencoder.add(Dense(14, activation='relu'))
autoencoder.add(Dense(X_train.shape[1], activation='sigmoid'))

# Compile the model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
autoencoder.fit(X_train, X_train, epochs=50, batch_size=256, validation_split=0.2)

# Predict on test set
X_test_pred = autoencoder.predict(X_test)

# Calculate reconstruction error
mse = np.mean(np.power(X_test - X_test_pred, 2), axis=1)
threshold = np.percentile(mse, 95)  # Set threshold for anomaly detection

# Classify based on threshold
y_pred_autoencoder = np.where(mse > threshold, 1, 0)

# Evaluation
print("Autoencoder Classification Report:")
print(classification_report(y_test, y_pred_autoencoder))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_autoencoder))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
15907/15907 ━━━━━━━━━━━━━━━━━━━━ 36s 2ms/step - loss: 0.6415 - val_loss: 0.6104
Epoch 2/50
15907/15907 ━━━━━━━━━━━━━━━━━━━━ 47s 3ms/step - loss: 0.6044 - val_loss: 0.6103
Epoch 3/50
15907/15907 ━━━━━━━━━━━━━━━━━━━━ 36s 2ms/step - loss: 0.6087 - val_loss: 0.6103
Epoch 4/50
15907/15907 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - loss: 0.6114 - val_loss: 0.6102
Epoch 5/50
15907/15907 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - loss: 0.6117 - val_loss: 0.6092
Epoch 6/50
15907/15907 ━━━━━━━━━━━━━━━━━━━━ 33s 2ms/step - loss: 0.6057 - val_loss: 0.6092
Epoch 7/50
15907/15907 ━━━━━━━━━━━━━━━━━━━━ 42s 2ms/step - loss: 0.6004 - val_loss: 0.6091
Epoch 8/50
15907/15907 ━━━━━━━━━━━━━━━━━━━━ 35s 2ms/step - loss: 0.6033 - val_loss: 0.6091
Epoch 9/50
15907/15907 ━━━━━━━━━━━━━━━━━━━━ 36s 2ms/step - loss: 0.6127 - val_loss: 0.6091
Epoch 10/50
15907/15907 ━━━━━━━━━━━━━━━━━━━━ 37s 2ms/step - loss: 0.6061 - val_loss: 0.6091
Epoch 11/50
15907/15907 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step - loss: 0.6074 - val_loss: 0.60

In [ ]:
# Evaluation
print("Optimized Autoencoder Classification Report:")
print(classification_report(y_test, y_pred_autoencoder))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_autoencoder))
print("Accuracy:", accuracy_score(y_test, y_pred_autoencoder))


Optimized Autoencoder Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.95      0.97   1270904
           1       0.00      0.18      0.01      1620

    accuracy                           0.95   1272524
   macro avg       0.50      0.57      0.49   1272524
weighted avg       1.00      0.95      0.97   1272524

Confusion Matrix:
 [[1207569   63335]
 [   1328     292]]
Accuracy: 0.9491852412999676
